In [12]:
import polars as pl
import pymysql
from sqlalchemy import create_engine


In [ ]:
#connect to https://relational.fel.cvut.cz/dataset/Financial with this below credentials
DB_CONFIG = {
    "host": "relational.fel.cvut.cz",
    "port": 3306,
    "user": "guest",
    "password": "ctu-relational",
    "database": "financial",
}

In [4]:
def create_mysql_engine(db_config):
    """Create a SQLAlchemy engine for MySQL."""
    connection_string = (
        f"mysql+pymysql://{db_config['user']}:{db_config['password']}@"
        f"{db_config['host']}:{db_config['port']}/{db_config['database']}"
    )
    return create_engine(connection_string, echo=False)

In [5]:
engine = create_mysql_engine(DB_CONFIG)

In [6]:
df = pl.read_database(
    query="SELECT * FROM account",
    connection=engine,
)

In [7]:
df

account_id,district_id,frequency,date
i64,i64,str,date
1,18,"""POPLATEK MESICNE""",1995-03-24
2,1,"""POPLATEK MESICNE""",1993-02-26
3,5,"""POPLATEK MESICNE""",1997-07-07
4,12,"""POPLATEK MESICNE""",1996-02-21
5,15,"""POPLATEK MESICNE""",1997-05-30
…,…,…,…
11333,8,"""POPLATEK MESICNE""",1994-05-26
11349,1,"""POPLATEK TYDNE""",1995-05-26
11359,61,"""POPLATEK MESICNE""",1994-10-01


In [13]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("SHOW TABLES"))
    for row in result:
        print(row) # Print each row of the result set

('account',)
('card',)
('client',)
('disp',)
('district',)
('loan',)
('order',)
('trans',)


In [14]:
df2 = pl.read_database(
    query="SELECT * FROM card",
    connection=engine,
)

In [15]:
df2

card_id,disp_id,type,issued
i64,i64,str,date
1,9,"""gold""",1998-10-16
2,19,"""classic""",1998-03-13
3,41,"""gold""",1995-09-03
4,42,"""classic""",1998-11-26
5,51,"""junior""",1995-04-24
…,…,…,…
1230,13312,"""classic""",1998-03-08
1233,13382,"""classic""",1996-07-06
1234,13386,"""classic""",1997-11-28


In [16]:
import os

import duckdb


In [19]:
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

In [ ]:
conn = duckdb.connect()
conn.execute(f"""
            CREATE SECRET berka_phoenixnguyen (
                TYPE s3,
                KEY_ID '{AWS_ACCESS_KEY_ID}',
                SECRET '{AWS_SECRET_ACCESS_KEY}',
                REGION 'ap-southeast-2',
                SCOPE 's3://berka-phoenixnguyen/' 
            );
        """)

In [23]:
conn.execute("""
            CREATE OR REPLACE TABLE account AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/raw/account.csv'
        """)

HTTPException: HTTP Error: Unable to connect to URL "https://berka-phoenixnguyen.s3.amazonaws.com/raw/account.csv": 404 (Not Found).

In [31]:
conn.execute("""
    SELECT * FROM account
""").df()
# need modification: frequency

,account_id,district_id,frequency,date
0,15,70,POPLATEK MESICNE,1993-10-02
1,24,74,POPLATEK MESICNE,1996-12-05
2,25,68,POPLATEK MESICNE,1996-07-28
3,35,75,POPLATEK MESICNE,1993-07-07
4,37,20,POPLATEK MESICNE,1997-08-18
...,...,...,...,...
4495,11027,32,POPLATEK TYDNE,1996-07-26
4496,11096,28,POPLATEK MESICNE,1996-12-10
4497,11186,12,POPLATEK MESICNE,1994-11-24
4498,11325,62,POPLATEK MESICNE,1996-10-15


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE card AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/raw/card.csv'
        """)
conn.execute("""
    SELECT * FROM card
""").df()
# no modification needed

,card_id,disp_id,type,issued
0,13,87,classic,1994-06-29
1,25,158,classic,1997-12-06
2,32,219,classic,1998-08-06
3,38,242,classic,1996-08-31
4,48,295,classic,1998-03-10
...,...,...,...,...
887,897,7576,classic,1998-01-18
888,1001,9247,classic,1998-11-20
889,1056,10220,classic,1997-11-06
890,1103,11189,classic,1995-11-04


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE client AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/raw/client.csv'
        """)
conn.execute("""
    SELECT * FROM client
""").df()
# no modification needed

,client_id,gender,birth_date,district_id
0,15,F,1918-08-28,21
1,21,F,1952-10-29,12
2,24,F,1939-07-29,43
3,43,M,1920-10-28,75
4,57,F,1961-12-17,21
...,...,...,...,...
5364,13750,F,1980-11-13,12
5365,13751,M,1979-07-20,12
5366,13803,M,1947-11-14,1
5367,13820,F,1948-07-31,33


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE disp AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/raw/disp.csv'
        """)
conn.execute("""
    SELECT * FROM disp
""").df()
# modification needed: type DISPONENT -> type USER

,disp_id,client_id,account_id,type
0,5,5,3,DISPONENT
1,8,8,6,OWNER
2,22,22,17,OWNER
3,24,24,18,OWNER
4,28,28,22,OWNER
...,...,...,...,...
5364,13512,13820,11244,OWNER
5365,13544,13852,11271,OWNER
5366,13607,13915,11320,OWNER
5367,13623,13931,11333,OWNER


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE district AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/raw/district.csv'
        """)
conn.execute("""
    SELECT * FROM district
""").df()
# no modification needed

,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677,99107
1,7,Melnik,central Bohemia,94725,38,28,1,3,6,63.4,9920,2.2,2.87,130,4289,4846
2,14,Ceske Budejovice,south Bohemia,177686,69,27,10,1,9,74.8,10045,1.4,1.71,135,6604,6295
3,25,Klatovy,west Bohemia,88757,60,33,3,2,10,61.7,8554,2.4,2.68,113,1822,2218
4,47,Pardubice,east Bohemia,162580,83,26,5,1,6,72.8,9538,1.5,1.81,111,6079,5410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,52,Usti nad Orlici,east Bohemia,139012,59,41,8,3,10,61.9,8363,2.5,3.49,108,2564,2799
73,53,Blansko,south Moravia,107911,88,33,6,2,7,50.9,8240,2.5,3.56,99,1850,1903
74,56,Breclav,south Moravia,124605,11,42,14,1,8,46.0,8772,4.6,4.98,126,2854,3158
75,62,Uherske Hradiste,south Moravia,145688,22,41,13,2,5,40.7,8544,1.2,1.86,109,2719,2906


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE loan AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/raw/loan.csv'
        """)
conn.execute("""
    SELECT * FROM loan
""").df()
# no modification needed

,loan_id,account_id,date,amount,duration,payments,status
0,4961,19,1996-04-29,30276,12,2523.0,B
1,4986,97,1997-08-10,102876,12,8573.0,A
2,4989,105,1998-12-05,352704,48,7348.0,C
3,4996,132,1996-11-06,88440,12,7370.0,A
4,5039,314,1997-05-03,66840,24,2785.0,C
...,...,...,...,...,...,...,...
677,7226,10940,1994-02-23,197748,36,5493.0,A
678,7253,11079,1997-04-26,98304,12,8192.0,A
679,7264,11141,1997-10-07,44940,12,3745.0,A
680,7292,11317,1998-11-22,317460,60,5291.0,C


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE orders AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/raw/orders.csv'
        """)
conn.execute("""
    SELECT * FROM orders
""").df()
# modifcation needed: k_symbol

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29408,4,UV,5848086,1285.0,SIPO
1,29448,35,MN,56788617,1022.0,SIPO
2,29460,39,WX,949460,2970.0,SIPO
3,29464,42,KL,55873340,239.0,None
4,29465,42,YZ,38318204,1592.0,POJISTNE
...,...,...,...,...,...,...
6466,45868,11065,MN,13170600,1445.0,UVER
6467,45885,11079,UV,40114671,8191.5,UVER
6468,45974,11138,YZ,73215025,3744.7,UVER
6469,46125,11231,CD,69176065,7440.5,UVER


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE trans AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/raw/trans.csv'
        """)
conn.execute("""
    SELECT * FROM trans
""").df()
# modification needed: k_symbol, type, operation

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,460655,1566,1994-02-11,VYDAJ,PREVOD NA UCET,948,21199,,OP,12450731
1,461109,1567,1998-01-09,VYDAJ,VYBER,1680,74776,None,None,<NA>
2,461149,1567,1997-09-18,PRIJEM,VKLAD,3600,26093,None,None,<NA>
3,461162,1567,1998-10-13,VYDAJ,VYBER,9200,58273,None,None,<NA>
4,461296,1568,1996-08-09,VYDAJ,PREVOD NA UCET,49,41958,,WX,99360131
...,...,...,...,...,...,...,...,...,...,...
1056315,914442,3116,1998-02-28,VYDAJ,VYBER,15,55720,SLUZBY,None,<NA>
1056316,914452,3116,1996-05-05,PRIJEM,VKLAD,3000,21918,None,None,<NA>
1056317,914723,3117,1994-12-19,VYDAJ,VYBER,3360,19902,None,None,<NA>
1056318,914797,3117,1997-05-31,VYDAJ,VYBER,15,19184,SLUZBY,None,<NA>


In [7]:
conn = duckdb.connect()

In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE account AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/processed/account.parquet'
        """)
conn.execute("""
    SELECT * FROM account
""").df()
# modification needed: k_symbol, type, operation

,account_id,district_id,frequency,date
0,83,26,MONTHLY ISSUANCE,1993-04-16
1,160,23,MONTHLY ISSUANCE,1993-09-04
2,199,70,MONTHLY ISSUANCE,1996-10-26
3,326,61,MONTHLY ISSUANCE,1997-08-06
4,349,72,MONTHLY ISSUANCE,1997-06-08
...,...,...,...,...
4495,6064,49,MONTHLY ISSUANCE,1997-01-11
4496,9203,70,MONTHLY ISSUANCE,1993-02-27
4497,9333,36,MONTHLY ISSUANCE,1996-11-08
4498,9646,1,WEEKLY ISSUANCE,1996-07-30


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE card AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/processed/card.parquet'
        """)
conn.execute("""
    SELECT * FROM card
""").df()
# modification needed: k_symbol, type, operation

,card_id,disp_id,type,issued
0,17,127,classic,1998-06-07
1,92,508,classic,1995-11-20
2,150,859,classic,1996-03-30
3,203,1231,classic,1998-09-16
4,210,1253,classic,1997-02-16
...,...,...,...,...
887,834,6301,classic,1998-08-21
888,847,6610,classic,1997-05-20
889,947,8366,classic,1997-11-08
890,1139,11836,junior,1996-03-26


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE client AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/processed/client.parquet'
        """)
conn.execute("""
    SELECT * FROM client
""").df()
# modification needed: k_symbol, type, operation

,client_id,gender,birth_date,district_id
0,99,M,1973-01-26,33
1,232,F,1921-06-17,55
2,355,M,1930-05-31,49
3,457,M,1950-09-26,70
4,466,F,1947-11-19,1
...,...,...,...,...
5364,10563,M,1949-01-07,68
5365,11337,M,1969-04-03,29
5366,12738,F,1960-10-29,23
5367,13406,F,1978-04-18,42


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE disp AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/processed/disp.parquet'
        """)
conn.execute("""
    SELECT * FROM disp
""").df()
# modification needed: k_symbol, type, operation

,disp_id,client_id,account_id,type
0,199,199,165,OWNER
1,355,355,292,USER
2,425,425,348,OWNER
3,468,468,387,OWNER
4,499,499,413,OWNER
...,...,...,...,...
5364,11183,11491,9333,OWNER
5365,11329,11637,9455,OWNER
5366,12108,12416,10093,OWNER
5367,12542,12850,10445,OWNER


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE district AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/processed/district.parquet'
        """)
conn.execute("""
    SELECT * FROM district
""").df()
# modification needed: k_symbol, type, operation

,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,53,Blansko,south Moravia,107911,88,33,6,2,7,50.9,8240,2.5,3.56,99,1850,1903
1,47,Pardubice,east Bohemia,162580,83,26,5,1,6,72.8,9538,1.5,1.81,111,6079,5410
2,73,Opava,north Moravia,182027,17,49,12,2,7,56.4,8746,3.3,3.74,90,4355,4433
3,10,Praha - vychod,central Bohemia,92084,55,29,4,3,5,46.7,10124,0.5,0.54,141,3810,4316
4,65,Znojmo,south Moravia,114200,101,41,4,1,4,43.7,8403,5.7,5.72,105,2157,2718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.6,5.05,109,5244,5892
73,21,Tabor,south Bohemia,103347,87,16,7,1,7,67.0,9104,1.5,2.07,123,2299,2354
74,27,Plzen - jih,west Bohemia,67298,71,19,10,0,7,43.8,8561,0.6,1.29,110,1029,1127
75,30,Sokolov,west Bohemia,94812,15,13,8,2,10,81.8,9650,3.3,3.67,100,2985,2804


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE loan AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/processed/loan.parquet'
        """)
conn.execute("""
    SELECT * FROM loan
""").df()

,loan_id,account_id,date,amount,duration,payments,status
0,5354,2000,1998-07-14,290820,60,4847.0,C
1,5683,3460,1998-05-29,140688,48,2931.0,C
2,5815,4116,1998-06-03,151200,36,4200.0,C
3,6185,5774,1995-11-28,31176,24,1299.0,A
4,6358,6715,1995-01-14,137880,60,2298.0,D
...,...,...,...,...,...,...,...
677,6486,7240,1998-01-15,152160,60,2536.0,C
678,6868,9208,1998-10-12,82128,24,3422.0,C
679,6969,9673,1998-09-12,88200,36,2450.0,C
680,6996,9828,1995-04-13,126600,24,5275.0,A


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE orders AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/processed/orders.parquet'
        """)
conn.execute("""
    SELECT * FROM orders
""").df()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29613,131,UV,29374742,6904.0,HOUSEHOLD
1,29678,183,AB,5552345,5202.0,HOUSEHOLD
2,29688,190,YZ,24279922,4519.0,HOUSEHOLD
3,29712,205,AB,99149345,4297.0,HOUSEHOLD
4,29722,209,EF,52778792,41.0,None
...,...,...,...,...,...,...
6466,43822,9730,AB,9366584,4764.0,None
6467,44012,9859,KL,56029701,765.0,INSURANCE
6468,44839,10404,UV,60165852,883.0,None
6469,44850,10411,GH,82482695,5142.0,LOAN


In [ ]:
conn.execute("""
            CREATE OR REPLACE TABLE trans AS
            SELECT DISTINCT * FROM 's3://berka-phoenixnguyen/processed/trans.parquet'
        """)
conn.execute("""
    SELECT * FROM trans
""").df()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,3145687,10440,1997-11-05,CREDIT,CREDIT IN CASH,7366,53136,None,None,<NA>
1,3147347,10445,1997-01-23,DEBIT,WITHDRAWAL IN CASH,3120,57129,None,None,<NA>
2,3182441,10558,1998-10-30,DEBIT,WITHDRAWAL IN CASH,2300,84408,HOUSEHOLD,None,0
3,3242341,10762,1998-02-12,DEBIT,REMITTANCE TO ANOTHER BANK,1912,34966,LOAN,YZ,96996306
4,3245111,10774,1998-07-19,DEBIT,WITHDRAWAL IN CASH,5880,25278,None,None,<NA>
...,...,...,...,...,...,...,...,...,...,...
1056315,2938673,9731,1998-04-14,DEBIT,WITHDRAWAL IN CASH,9500,17993,None,None,<NA>
1056316,2966942,9826,1997-06-19,DEBIT,WITHDRAWAL IN CASH,7400,24943,None,None,<NA>
1056317,3021420,10018,1997-04-13,DEBIT,WITHDRAWAL IN CASH,10400,21424,None,None,<NA>
1056318,3036235,10065,1996-09-10,DEBIT,WITHDRAWAL IN CASH,50500,66336,None,None,<NA>
